# Generate and send daily sales report from the Items and Sales data

In [1]:
##### CHANGE TO TIME DELTA - 1 !!!!!!!

import datetime
from datetime import timedelta
 
sales_date = datetime.datetime.today() - timedelta(days=1)

week_day=sales_date.strftime('%A') 
day_no=sales_date.day

print(week_day, day_no)

Saturday 16


In [2]:
import pandas as pd # data manipulation


_sales = pd.read_excel('Sales.xls')
_items = pd.read_excel('Items.xls')

ayla_sales = pd.read_excel('AylaSales.xls')
ayla_items = pd.read_excel('AylaItems.xls')

sales = pd.concat([_sales,ayla_sales])
items = pd.concat([_items,ayla_items])

branches_dict = {
    2:'Abdoun',
    3:'Khalda',
    4:'Wadi Saqra',
    5:'Airport',
    6:'Warehouse1',
    7:'J.Amman',
    9:'Insurance',
    10:'Abdoun 2',
    11:'Madinah 1',
    12:'Fifth Circle',
    14:'Shmesani 1',
    15:'Rabieah',
    16:'Warehouse2',
    17:'Madinah 2',
    18:'Damged',
    19:'Tender',
    20:'Taybah',
    23:'Specialist 1',
    24:'Yadodah',
    25:'Specialist 2',
    26:'Malyah 2',
    28:'Wehdat',
    29:'Yasmin',
    30:'Ghosheh St.',
    34:'Store-Yasmin',
    35:'Wrong Stock',
    36:'Dabouq',
    38:'NO STOCK',
    39:'Damged2',
    40:'Tabarbour',
    41:'On Hold'
   }





# Data
df = pd.DataFrame()
# Column for branches
df['Branches'] = [branch for branch in branches_dict.values()]

In [3]:
sales.rename(columns={'VHFNo':'VHFNO'}, inplace=True)

In [4]:
sales['P.Key'] = (sales['PosNo'].astype(str) + ' ' +
                  sales['VHFDate'].astype(str) + ' ' +
                  sales['VHFNO'].astype(str) + ' ' +
                  sales['USERNAME'].astype(str) + ' ' +
                  sales['TYPE'].astype(str)
                 )

In [5]:
items['P.Key'] = (items['POSNO'].astype(str) + ' ' +
                  items['VHFDATE'].astype(str) + ' ' +
                  items['VHFNO'].astype(str) + ' ' +
                  items['USERNAME'].astype(str) + ' ' +
                  items['TYPE'].astype(str)
                 )

In [6]:
items['Net Sales'] = items['TOTAL'] - items['DISCOUNT'] - items['TAXAMT']

In [7]:
# Function that returns sums of a measure for each branch
# metric: column such as Total Sales, Discount
# sales_type: Choosing type of sale (Sales by default)
# return: list of sums
def get_sums(metric, sales_type='Sales'):
    list_of_metric = []
    # temporary table filtering sales type
    temp=items.loc[items['TYPE'] == sales_type]
    # Going through each branch number
    for i in branches_dict.keys():
        tempo=temp.loc[temp['POSNO'] == i]
        # Add sum of measure to a list
        list_of_metric.append(sum(tempo[metric]))
    return list_of_metric

def get_net_sums(metric):
    list_of_metric = []
    # Going through each branch number
    for i in branches_dict.keys():
        tempo=items.loc[items['POSNO'] == i]
        # Add sum of measure to a list
        list_of_metric.append(sum(tempo[metric]))
    return list_of_metric

# Generate the following columns' sums
Total_Sales = get_sums('TOTAL')
Returns = get_sums('TOTAL', sales_type='Return Sales')
Discount = get_net_sums('DISCOUNT')
#Discount_Returns = get_sums('DISCOUNT', sales_type='Return Sales')
Taxes = get_net_sums('TAXAMT')
#Taxes_Returns = get_sums('TAXAMT', sales_type='Return Sales')
Net_Sales = get_net_sums('Net Sales')


In [8]:
# Get number of invoices for each branch
number_of_invoices_for_each_branch = []
# temporary table filtering sales type
temp_sales=sales.loc[sales['TYPE'] == 'Sales']
temp_rets=sales.loc[sales['TYPE'] == 'Return Sales']
# Going through each branch number
for i in branches_dict.keys():
    tempo=temp_sales.loc[temp_sales['PosNo'] == i]
    tempoo=temp_rets.loc[temp_rets['PosNo'] == i]
    sales_len = len(tempo['P.Key'].unique())
    rets_len = len(tempoo['P.Key'].unique())
    num_of_invoices = sales_len - rets_len
    # Extract number of invoices
    number_of_invoices_for_each_branch.append(num_of_invoices)

# Function that adds to created table the metrics
def add_to_table(variable, column_name):
    df[column_name] = [sums for sums in variable]

In [9]:
# Formualate the table
add_to_table(Total_Sales, 'Total Sales')
add_to_table(Returns, 'Returns')
df['Returns'] = abs(df['Returns'])
df['Return Ratio'] = df['Returns']/df['Total Sales']
add_to_table(Discount, 'Discount')
df['Discount Ratio'] = df['Discount']/df['Total Sales']
add_to_table(Taxes, 'Taxes')
add_to_table(Net_Sales, 'Net Sales')
add_to_table(number_of_invoices_for_each_branch, '# of Invoices')
df['Avg Invoice'] = df['Net Sales']/df['# of Invoices']

In [10]:
new_df = pd.DataFrame(columns=[df.columns])

In [11]:
branch_list=['Khalda','Abdoun','J.Amman 2','Madinah 2','Fifth Circle', 'Specialist 1',
            'Abdoun 2', 'Yasmin', 'Wadi Saqra', 'Taybah', 'Rabieah', 'Shmesani 1','Yadodah',
            'Madinah 1', 'Ghosheh St.', 'J.Amman 3', 'Wehdat', 'Specialist 2', 'Dabouq',
            'Malyah 2', 'Abdoun 3', 'J.Amman 4', 'Airport', 'Ayla','Tabarbour','Airport 2','ZAIN BRANCH','ISRA HOSPITAL BRANCH',
            'Tender', 'Insurance', 'Warehouse1', 'NO STOCK','Rawhi Wellness']

In [12]:
new_df=df[df['Branches'] == 'J.Amman']
for branch in branch_list:
    new_df = pd.concat([new_df, df[df['Branches'] == branch]])

In [13]:
newest_df = new_df.reset_index()

In [14]:
cards=['كابيتال بنك -بطاقات ائتمان','كابيتال بنك -بطاقات ائتمان','كابيتال بنك -بطاقات ائتمان','MEPS','AMERICAN EXPRESS','موبي كاش العربي',
      'CAPITAL BANK -CREDIT CARD','ARAB ISLAMIC INTERNATIONAL BANK','AMEX-NO STOCK', 'CAPITAL BANK - CREDIT CARD - NO STOCK',
      'MOBI CASH -NO STOCK','شركة الشرق الاوسط لخدمات الدفع-MEPS','MEPS -NO STOCK','امريكان اكسبرس',
      'MOBI CASH ARAB BANK','MOBI CASH  -NO STOCK','MOBI CASH ARB BANK', 'كابحتاء بؤك -بطاقات ائتأاؤ', 'MEPS-NO STOCK']

sales_details = pd.DataFrame(columns=['Cash', 'Insurance', 'Credit(cards)', 'Credit(customers)'])

In [15]:
# Filling sales details to compare between cash, credit and insurance

# Merge items and sales 
inner_join = pd.merge(items, 
                      sales, 
                      on ='P.Key', 
                      how ='inner')


In [16]:
# Adds net sales to a column
inner_join['Net Sales'] = inner_join['TOTAL_x'] - inner_join['DISCOUNT'] - inner_join['TAXAMT']

In [17]:
cash_temp=inner_join.loc[inner_join['PAYTYPE'] == 'Cash']
cash_sum = sum(cash_temp['Net Sales'])

In [18]:
cash_temp=inner_join.loc[inner_join['PAYTYPE'] == 'Cash']
cash_sum = sum(cash_temp['Net Sales'])

ins_temp=inner_join.loc[(inner_join['PAYTYPE'] == 'Insurance') |
                        (inner_join['PAYTYPE'] == 'Cash,Insurance') |
                        (inner_join['PAYTYPE'] == 'Credit,Insurance') |
                        (inner_join['PAYTYPE'] == 'Cash,Credit,Insurance')]
ins_sum=sum(ins_temp['Net Sales'])

credit=inner_join.loc[(inner_join['PAYTYPE'] == 'Credit') |
                      (inner_join['PAYTYPE'] == 'Cash,Credit')]



In [19]:
credit_new = credit.reset_index()

In [20]:
credit_cust_sums = 0
credit_cards_sums = 0
leng = len(credit_new)
for i in range(0,leng):
    row = credit_new.iloc[i]
    if row['CUSTOMERNAME'] in cards:
        credit_cards_sums = credit_cards_sums + credit_new['Net Sales'][i]
    else:
        credit_cust_sums = credit_cust_sums + credit_new['Net Sales'][i]

In [21]:
sales_details['Cash'] = [cash_sum]
sales_details['Insurance'] = [ins_sum]
sales_details['Credit(cards)'] = [credit_cards_sums]
sales_details['Credit(customers)'] = [credit_cust_sums]

In [22]:
tax_dets = inner_join.groupby('TAXABLE')['Net Sales'].sum()

In [23]:
tax_details = pd.DataFrame(tax_dets)

### Writing to Excel Sheet

In [24]:
if week_day == 'Saturday':
    ts_column = 'C'
    ret_column = 'D'
    disc_column = 'F'
    tax_column = 'H'
    inv_column = 'L'
    if day_no == 1:
        row = 3
    elif (day_no > 1) & (day_no < 9):
        row=40
    elif (day_no > 8) & (day_no < 16):
        row=77
    elif (day_no > 15) & (day_no < 23):
        row=114
    elif (day_no > 22) & (day_no < 30):
        row=151
    elif day_no > 29:
        row=188
        
elif week_day == 'Sunday':
    ts_column = 'Q'
    ret_column = 'R'
    disc_column = 'T'
    tax_column = 'V'
    inv_column = 'Z'
    if (day_no > 0) & (day_no < 3):
        row = 3
    elif (day_no > 2) & (day_no < 10):
        row=40
    elif (day_no > 9) & (day_no < 17):
        row=77
    elif (day_no > 16) & (day_no < 24):
        row=114
    elif (day_no > 23) & (day_no < 31):
        row=151
    elif day_no > 30:
        row=188
        
elif week_day == 'Monday':
    ts_column = 'AE'
    ret_column = 'AF'
    disc_column = 'AH'
    tax_column = 'AJ'
    inv_column = 'AN'
    if (day_no > 0) & (day_no < 4):
        row = 3
    elif (day_no > 3) & (day_no < 11):
        row=40
    elif (day_no > 10) & (day_no < 18):
        row=77
    elif (day_no > 17) & (day_no < 25):
        row=114
    elif (day_no > 24) & (day_no < 32):
        row=151

elif week_day == 'Tuesday':
    ts_column = 'AS'
    ret_column = 'AT'
    disc_column = 'AV'
    tax_column = 'AX'
    inv_column = 'BB'
    if (day_no > 0) & (day_no < 5):
        row = 3
    elif (day_no > 4) & (day_no < 12):
        row=40
    elif (day_no > 11) & (day_no < 19):
        row=77
    elif (day_no > 18) & (day_no < 26):
        row=114
    elif (day_no > 25) & (day_no < 32):
        row=151
        
elif week_day == 'Wednesday':
    ts_column = 'BG'
    ret_column = 'BH'
    disc_column = 'BJ'
    tax_column = 'BL'
    inv_column = 'BP'
    if (day_no > 0) & (day_no < 6):
        row = 3
    elif (day_no > 5) & (day_no < 13):
        row=40
    elif (day_no > 12) & (day_no < 20):
        row=77
    elif (day_no > 19) & (day_no < 27):
        row=114
    elif (day_no > 26) & (day_no < 32):
        row=151
        
elif week_day == 'Thursday':
    ts_column = 'BU'
    ret_column = 'BV'
    disc_column = 'BX'
    tax_column = 'BZ'
    inv_column = 'CD'
    if (day_no > 0) & (day_no < 7):
        row = 3
    elif (day_no > 6) & (day_no < 14):
        row=40
    elif (day_no > 13) & (day_no < 21):
        row=77
    elif (day_no > 20) & (day_no < 28):
        row=114
    elif (day_no > 27) & (day_no < 32):
        row=151
        
elif week_day == 'Friday':
    ts_column = 'CI'
    ret_column = 'CJ'
    disc_column = 'CL'
    tax_column = 'CN'
    inv_column = 'CR'
    if (day_no > 0) & (day_no < 8):
        row = 3
    elif (day_no > 7) & (day_no < 15):
        row=40
    elif (day_no > 14) & (day_no < 22):
        row=114
    elif (day_no > 21) & (day_no < 29):
        row=151
    elif (day_no > 28) & (day_no < 32):
        row=188

### Writing Details

In [25]:
pay_cash_column = 'AN'
pay_ins_column = 'AO'
pay_card_column = 'AP'
pay_cust_column = 'AQ'

tax_0_column = 'AU'
tax_1_column = 'AV'
tax_4_column = 'AW'
tax_7_column = 'AX'
tax_16_column = 'AY'

sales_ts_column = 'BC'
sales_ret_column = 'BD'
sales_tax_column = 'BE'
sales_disc_column = 'BF'

details_starting_row = 230

In [26]:
detail_row = details_starting_row + (day_no-1)
detail_row

245

In [27]:
import openpyxl as xl
daily_sales_file = xl.load_workbook('Daily Sales Report.xlsx')
sheet = daily_sales_file[sales_date.strftime('%B')]

idx=0
for i in range(row, row + 32):
    sheet[ts_column+str(i)] = newest_df['Total Sales'][idx]
    sheet[ret_column+str(i)] = newest_df['Returns'][idx]
    sheet[disc_column+str(i)] = newest_df['Discount'][idx]
    sheet[tax_column+str(i)] = newest_df['Taxes'][idx]
    sheet[inv_column+str(i)] = newest_df['# of Invoices'][idx]
    idx += 1

In [28]:
sheet[pay_cash_column+str(detail_row)] = sales_details['Cash'][0]
sheet[pay_ins_column+str(detail_row)] = sales_details['Insurance'][0]
sheet[pay_card_column+str(detail_row)] = sales_details['Credit(cards)'][0]
sheet[pay_cust_column+str(detail_row)] = sales_details['Credit(customers)'][0]

In [29]:
if 0 in tax_details.index:
    sheet[tax_0_column+str(detail_row)] = tax_details['Net Sales'][0]
if 1 in tax_details.index:
    sheet[tax_1_column+str(detail_row)] = tax_details['Net Sales'][1]
sheet[tax_4_column+str(detail_row)] = tax_details['Net Sales'][4]
if 7 in tax_details.index:
    sheet[tax_7_column+str(detail_row)] = tax_details['Net Sales'][7]
sheet[tax_16_column+str(detail_row)] = tax_details['Net Sales'][16]

In [30]:
sheet[sales_ts_column+str(detail_row)] = sum(newest_df['Total Sales'])
sheet[sales_ret_column+str(detail_row)] = sum(newest_df['Returns'])
sheet[sales_tax_column+str(detail_row)] = sum(newest_df['Taxes'])
sheet[sales_disc_column+str(detail_row)] = sum(newest_df['Discount'])

In [31]:
daily_sales_file.save('Daily Sales Report.xlsx')

### Target Achieved All

In [32]:
target_ach_file = xl.load_workbook('Branches monthly target - 2023 All Branches.xlsx')
sheet = target_ach_file[sales_date.strftime('%b')]

In [33]:
if day_no < 8:
    day_column = {1:'A', 2:'B', 3:'C', 4:'D', 5:'E', 6:'F', 7:'G'}
    all_row = 2
elif (day_no > 7) & (day_no < 15):
    day_column = {8:'A', 9:'B', 10:'C', 11:'D', 12:'E', 13:'F', 14:'G'}
    all_row = 4
elif (day_no > 14) & (day_no < 22):
    day_column = {15:'A', 16:'B', 17:'C', 18:'D', 19:'E', 20:'F', 21:'G'}
    all_row = 6
elif (day_no > 21) & (day_no < 29):
    day_column = {22:'A', 23:'B', 24:'C', 25:'D', 26:'E', 27:'F', 28:'G'}
    all_row = 8
else:
    day_column = {29:'A', 30:'B', 31:'C'}
    all_row = 10
    
get_column = day_column[day_no]

In [34]:
sheet[get_column+str(all_row)] = sum(newest_df.loc[newest_df.index < 27]['Net Sales'])

In [35]:
import calendar
sheet['G12'] = '='+get_column+str(all_row)+'/($A$12/'+str(calendar.monthrange(sales_date.year, sales_date.month)[1])+')'

In [36]:
sheet['H11'] = day_no

In [37]:
target_ach_file.save('Branches monthly target - 2023 All Branches.xlsx')

### Branch Managers

#### Dr. Dr.3

##### Monthly Target Sheet

In [38]:
dr_Dr.3_target = xl.load_workbook('Branches monthly target - 2023 Dr. Dr.3.xlsx')
sheet = dr_Dr.3_target[sales_date.strftime('%b')]

In [39]:
if day_no < 8:
    day_column = {1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H'}
    dr_Dr.3_row = 2
elif (day_no > 7) & (day_no < 15):
    day_column = {8:'B', 9:'C', 10:'D', 11:'E', 12:'F', 13:'G', 14:'H'}
    dr_Dr.3_row = 10
elif (day_no > 14) & (day_no < 22):
    day_column = {15:'B', 16:'C', 17:'D', 18:'E', 19:'F', 20:'G', 21:'H'}
    dr_Dr.3_row = 18
elif (day_no > 21) & (day_no < 29):
    day_column = {22:'B', 23:'C', 24:'D', 25:'E', 26:'F', 27:'G', 28:'H'}
    dr_Dr.3_row = 26
else:
    day_column = {29:'B', 30:'C', 31:'D'}
    dr_Dr.3_row = 34
    
get_column = day_column[day_no]
dr_Dr.3_branches = ['Abdoun', 'Abdoun 2', 'Abdoun 3', 'Fifth Circle', 'Wadi Saqra']

In [40]:
for branch in dr_Dr.3_branches:
    dr_Dr.3_net_sales = newest_df.loc[newest_df['Branches'] == branch]['Net Sales'].values[0]
    sheet[get_column+str(dr_Dr.3_row)] = dr_Dr.3_net_sales
    dr_Dr.3_row = dr_Dr.3_row+1

In [41]:
sheet['L1'] = day_no

In [42]:
dr_Dr.3_target.save('Branches monthly target - 2023 Dr. Dr.3.xlsx')

##### Avg Invoice Sheet

In [43]:
dr_Dr.3_avg_inv = xl.load_workbook('Branches Avg Invoice - 2023 Dr. Dr.3.xlsx')
sheet_avg = dr_Dr.3_avg_inv[sales_date.strftime('%b')]

In [44]:
if day_no < 8:
    day_column = {1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H'}
    dr_Dr.3_row = 2
elif (day_no > 7) & (day_no < 15):
    day_column = {8:'B', 9:'C', 10:'D', 11:'E', 12:'F', 13:'G', 14:'H'}
    dr_Dr.3_row = 10
elif (day_no > 14) & (day_no < 22):
    day_column = {15:'B', 16:'C', 17:'D', 18:'E', 19:'F', 20:'G', 21:'H'}
    dr_Dr.3_row = 18
elif (day_no > 21) & (day_no < 29):
    day_column = {22:'B', 23:'C', 24:'D', 25:'E', 26:'F', 27:'G', 28:'H'}
    dr_Dr.3_row = 26
else:
    day_column = {29:'B', 30:'C', 31:'D'}
    dr_Dr.3_row = 34
    
get_column = day_column[day_no]

In [45]:
for branch in dr_Dr.3_branches:
    dr_Dr.3_avg_inv_no = newest_df.loc[newest_df['Branches'] == branch]['Avg Invoice'].values[0]
    sheet_avg[get_column+str(dr_Dr.3_row)] = dr_Dr.3_avg_inv_no
    dr_Dr.3_row = dr_Dr.3_row+1

In [46]:
dr_Dr.3_avg_inv.save('Branches Avg Invoice - 2023 Dr. Dr.3.xlsx')

#### Dr. Dr.2

##### Monthly Target

In [47]:
dr_Dr.2_target = xl.load_workbook('Branches monthly target - 2023 Dr. Dr.2.xlsx')
sheet = dr_Dr.2_target[sales_date.strftime('%b')]

dr_Dr.2_branches = ['Ghosheh St.', 'Madinah 1', 'Madinah 2', 'Malyah 2', 'Taybah', 'Wehdat', 'Yadodah', 'Yasmin']

In [48]:
if day_no < 8:
    dr_Dr.2_row = 2
elif (day_no > 7) & (day_no < 15):
    dr_Dr.2_row = 13
elif (day_no > 14) & (day_no < 22):
    dr_Dr.2_row = 24
elif (day_no > 21) & (day_no < 29):
    dr_Dr.2_row = 35
else:
    dr_Dr.2_row = 46

In [49]:
for branch in dr_Dr.2_branches:
    dr_Dr.2_net_sales = newest_df.loc[newest_df['Branches'] == branch]['Net Sales'].values[0]
    sheet[get_column+str(dr_Dr.2_row)] = dr_Dr.2_net_sales
    dr_Dr.2_row = dr_Dr.2_row+1

In [50]:
sheet['L1'] = day_no

In [51]:
dr_Dr.2_target.save('Branches monthly target - 2023 Dr. Dr.2.xlsx')

##### Avg Invoice

In [52]:
dr_Dr.2_avg_inv = xl.load_workbook('Branches Avg Invoice - 2023 Dr. Dr.2.xlsx')
sheet_avg = dr_Dr.2_avg_inv[sales_date.strftime('%b')]

dr_Dr.2_branches = ['Ghosheh St.', 'Madinah 1', 'Madinah 2', 'Malyah 2', 'Taybah', 'Wehdat', 'Yadodah', 'Yasmin']

In [53]:
if day_no < 8:
    dr_Dr.2_row = 2
elif (day_no > 7) & (day_no < 15):
    dr_Dr.2_row = 13
elif (day_no > 14) & (day_no < 22):
    dr_Dr.2_row = 24
elif (day_no > 21) & (day_no < 29):
    dr_Dr.2_row = 35
else:
    dr_Dr.2_row = 46

In [54]:
for branch in dr_Dr.2_branches:
    dr_Dr.2_avg_inv_no = newest_df.loc[newest_df['Branches'] == branch]['Avg Invoice'].values[0]
    sheet_avg[get_column+str(dr_Dr.2_row)] = dr_Dr.2_avg_inv_no
    dr_Dr.2_row = dr_Dr.2_row+1

In [55]:
dr_Dr.2_avg_inv.save('Branches Avg Invoice - 2023 Dr. Dr.2.xlsx')

#### Dr. Dr.1

##### Monthly Target

In [56]:
dr_Dr.1_target = xl.load_workbook('Branches monthly target - 2023 Dr. Dr.1.xlsx')
sheet = dr_Dr.1_target[sales_date.strftime('%b')]

dr_Dr.1_branches = ['Airport', 'Airport 2', 'Dabouq',
                     'Khalda', 'Rabieah', 'Shmesani 1', 'Tabarbour',
                     'Specialist 1', 'Specialist 2']

In [57]:
if day_no < 8:
    dr_Dr.1_row = 2
elif (day_no > 7) & (day_no < 15):
    dr_Dr.1_row = 17
elif (day_no > 14) & (day_no < 22):
    dr_Dr.1_row = 32
elif (day_no > 21) & (day_no < 29):
    dr_Dr.1_row = 47
else:
    dr_Dr.1_row = 62

In [58]:
for branch in dr_Dr.1_branches:
    dr_Dr.1_net_sales = newest_df.loc[newest_df['Branches'] == branch]['Net Sales'].values[0]
    sheet[get_column+str(dr_Dr.1_row)] = dr_Dr.1_net_sales
    dr_Dr.1_row = dr_Dr.1_row+1

In [59]:
sheet['L1'] = day_no

In [60]:
dr_Dr.1_target.save('Branches monthly target - 2023 Dr. Dr.1.xlsx')

##### Avg Invoice

In [61]:
dr_Dr.1_avg_inv = xl.load_workbook('Branches Avg Invoice - 2023 Dr. Dr.1.xlsx')
sheet_avg = dr_Dr.1_avg_inv[sales_date.strftime('%b')]

dr_Dr.1_branches = ['Airport', 'Airport 2', 'Dabouq',
                     'Khalda', 'Rabieah', 'Shmesani 1', 'Tabarbour',
                     'Specialist 1', 'Specialist 2']

In [62]:
if day_no < 8:
    dr_Dr.1_row = 2
elif (day_no > 7) & (day_no < 15):
    dr_Dr.1_row = 17
elif (day_no > 14) & (day_no < 22):
    dr_Dr.1_row = 32
elif (day_no > 21) & (day_no < 29):
    dr_Dr.1_row = 47
else:
    dr_Dr.1_row = 62

In [63]:
for branch in dr_Dr.1_branches:
    dr_Dr.1_avg_inv_no = newest_df.loc[newest_df['Branches'] == branch]['Avg Invoice'].values[0]
    sheet_avg[get_column+str(dr_Dr.1_row)] = dr_Dr.1_avg_inv_no
    dr_Dr.1_row = dr_Dr.1_row+1

In [64]:
dr_Dr.1_avg_inv.save('Branches Avg Invoice - 2023 Dr. Dr.1.xlsx')

#### Dr. Sana'

##### Monthly Target

In [65]:
dr_sana_target = xl.load_workbook('Branches monthly target - 2023 Dr. Dr.4.xlsx')
sheet = dr_sana_target[sales_date.strftime('%b')]

dr_sana_branches = ['J.Amman', 'J.Amman 2', 'J.Amman 3', 'J.Amman 4']

In [66]:
if day_no < 8:
    dr_sana_row = 2
elif (day_no > 7) & (day_no < 15):
    dr_sana_row = 9
elif (day_no > 14) & (day_no < 22):
    dr_sana_row = 16
elif (day_no > 21) & (day_no < 29):
    dr_sana_row = 23
else:
    dr_sana_row = 30

In [67]:
for branch in dr_sana_branches:
    dr_sana_net_sales = newest_df.loc[newest_df['Branches'] == branch]['Net Sales'].values[0]
    sheet[get_column+str(dr_sana_row)] = dr_sana_net_sales
    dr_sana_row = dr_sana_row+1

In [68]:
sheet['L1'] = day_no

In [69]:
dr_sana_target.save('Branches monthly target - 2023 Dr. Dr.4.xlsx')

##### Avg Invoice

In [70]:
dr_sana_avg_inv = xl.load_workbook('Branches Avg Invoice - 2023 Dr. Dr.4.xlsx')
sheet_avg = dr_sana_avg_inv[sales_date.strftime('%b')]

dr_sana_branches = ['J.Amman', 'J.Amman 2', 'J.Amman 3', 'J.Amman 4']

In [71]:
if day_no < 8:
    dr_sana_row = 2
elif (day_no > 7) & (day_no < 15):
    dr_sana_row = 9
elif (day_no > 14) & (day_no < 22):
    dr_sana_row = 16
elif (day_no > 21) & (day_no < 29):
    dr_sana_row = 23
else:
    dr_sana_row = 30

In [72]:
for branch in dr_sana_branches:
    dr_sana_avg_inv_no = newest_df.loc[newest_df['Branches'] == branch]['Avg Invoice'].values[0]
    sheet_avg[get_column+str(dr_sana_row)] = dr_sana_avg_inv_no
    dr_sana_row = dr_sana_row+1

In [73]:
dr_sana_avg_inv.save('Branches Avg Invoice - 2023 Dr. Dr.4.xlsx')